In [1]:
!pip install wordninja
!pip install fuzzywuzzy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 541.6/541.6 kB 7.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for wordninja: filename=wordninja-2.0.0-py3-none-any.whl size=541530 sha256=55b37317dd9c02957a029283e03bbeaf9a7976ae23bf6ae95bbca207ab3e8625
  Stored in directory: /root/.cache/pip/wheels/e6/66/9c/712044a983337f5d44f90abcd244bd4b8ad28ee64750404b50
Successfully built wordninja


In [ ]:
import pandas as pd
import re
import wordninja  # Install via pip install wordninja

# Add a custom dictionary to handle specific cases like 'Baler'
custom_words = ['baler']  # Add any other problematic words here

# Function to format text
def format_text(text):
    if isinstance(text, str):  # Ensure the value is a string
        # Step 1: Convert to lowercase
        text = text.lower()

        # Step 2: Replace dots and underscores with spaces
        text = text.replace('.', ' ').replace('_', ' ')

        # Step 3: Add spaces after punctuation marks like :, ;, -, /, etc.
        text = re.sub(r'([:,;\/-])([a-zA-Z0-9])', r'\1 \2', text)

        # Step 4: Split concatenated words using wordninja
        words = []
        for token in text.split():
            # Use wordninja to split concatenated words, checking for custom words first
            if token in custom_words:
                words.append(token)  # If it's a custom word, add it as is
            else:
                split_words = wordninja.split(token)
                words.extend(split_words)

        # Join the split words back into a single string
        text = ' '.join(words)

        # Step 5: Capitalize the first letter of each sentence
        text = '. '.join(sentence.strip().capitalize() for sentence in text.split('.'))

        # Step 6: Remove any remaining extra spaces
        text = ' '.join(text.split())

    return text

# Read the Excel file
file_path = '/content/DA - Task 1..xlsx'

# Read both sheets
df_task = pd.read_excel(file_path, sheet_name='Task')
df_taxonomy = pd.read_excel(file_path, sheet_name='Taxonomy')

# Apply formatting to the specified columns in the 'Task' sheet
columns_to_format = ['Product Category', 'Complaint', 'Cause', 'Correction']
for column in columns_to_format:
    df_task[column] = df_task[column].apply(format_text)

# Save both sheets ('Task' and 'Taxonomy') to a new Excel file
output_file_path = 'Cleanedexltask1.xlsx'
with pd.ExcelWriter(output_file_path, engine='openpyxl') as writer:
    df_task.to_excel(writer, sheet_name='Task', index=False)
    df_taxonomy.to_excel(writer, sheet_name='Taxonomy', index=False)

print(f"Formatted data saved to {output_file_path}")

Formatted data saved to Cleanedexltask1.xlsx


In [ ]:
import pandas as pd
from fuzzywuzzy import process

# Load the Excel file
file_path = "/content/Cleanedexltask1.xlsx"  # Update with your file path
task_sheet = pd.read_excel(file_path, sheet_name="Task")
taxonomy_sheet = pd.read_excel(file_path, sheet_name="Taxonomy")

# Convert Taxonomy sheet to a dictionary
taxonomy = {
    "Root Cause": list(taxonomy_sheet["Root Cause"].dropna()),
    "Symptom Condition": list(taxonomy_sheet["Symptom Condition "].dropna()),
    "Symptom Component": list(taxonomy_sheet["Symptom Component"].dropna()),
    "Fix Condition": list(taxonomy_sheet["Fix Condition"].dropna()),
    "Fix Component": list(taxonomy_sheet["Fix Component"].dropna())
}

# Function to preprocess text
def preprocess_text(text):
    if pd.isna(text) or str(text).strip() == "":
        return ""
    # Convert to lowercase and remove punctuation
    text = str(text).lower().replace(",", "").replace(".", "").replace(":", "")
    return text

# Function to extract matches using fuzzy matching
def extract_matches(text, category, limit=3):
    if pd.isna(text) or str(text).strip() == "":
        return []
    matches = process.extract(preprocess_text(text), [preprocess_text(term) for term in taxonomy[category]], limit=20)
    filtered = []
    seen = set()
    for match, score in matches:
        original_match = taxonomy[category][[preprocess_text(term) for term in taxonomy[category]].index(match)]
        if score >= 60 and original_match not in seen:  # Lower threshold to capture more matches
            filtered.append(original_match)
            seen.add(original_match)
        if len(filtered) == limit:
            break
    return filtered

# Ensure all required columns exist in the DataFrame
required_columns = [
    "Primary Key", "Order Date", "Product Category", "Complaint", "Cause", "Correction",
    "Root Cause", "Symptom Condition 1", "Symptom Component 1", "Symptom Condition 2",
    "Symptom Component 2", "Symptom Condition 3", "Symptom Component 3", "Fix Condition 1",
    "Fix Component 1", "Fix Condition 2", "Fix Component 2", "Fix Condition 3", "Fix Component 3"
]
for col in required_columns:
    if col not in task_sheet.columns:
        task_sheet[col] = ""

# Extract Root Cause
task_sheet["Root Cause"] = task_sheet["Cause"].apply(
    lambda x: extract_matches(str(x), "Root Cause", 1)[0] if extract_matches(str(x), "Root Cause", 1) else ""
)

# Extract Symptoms (Condition and Component)
symptom_conditions = task_sheet["Complaint"].apply(lambda x: extract_matches(str(x), "Symptom Condition"))
symptom_components = task_sheet["Complaint"].apply(lambda x: extract_matches(str(x), "Symptom Component"))

# Populate Symptom Columns
for i in range(1, 4):  # Loop through 1 to 3
    task_sheet[f"Symptom Condition {i}"] = symptom_conditions.apply(
        lambda x, idx=i-1: x[idx] if len(x) > idx else ""
    )
    task_sheet[f"Symptom Component {i}"] = symptom_components.apply(
        lambda x, idx=i-1: x[idx] if len(x) > idx else ""
    )

# Extract Fixes (Condition and Component)
fix_conditions = task_sheet["Correction"].apply(lambda x: extract_matches(str(x), "Fix Condition"))
fix_components = task_sheet["Correction"].apply(lambda x: extract_matches(str(x), "Fix Component"))

# Populate Fix Columns
for i in range(1, 4):  # Loop through 1 to 3
    task_sheet[f"Fix Condition {i}"] = fix_conditions.apply(
        lambda x, idx=i-1: x[idx] if len(x) > idx else ""
    )
    task_sheet[f"Fix Component {i}"] = fix_components.apply(
        lambda x, idx=i-1: x[idx] if len(x) > idx else ""
    )

# Save the updated DataFrame to a new Excel file
output_file_path = "Final_Task1.xlsx"
task_sheet.to_excel(output_file_path, index=False)
print(f"Processing complete. Output saved to '{output_file_path}'.")

/usr/local/lib/python3.11/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


Processing complete. Output saved to 'Final_Task1.xlsx'.
